In [1]:
import numpy as np
import os
import math
import matplotlib.pyplot as plt
import glob
import sys
from astroquery.jplhorizons import Horizons

In [2]:
### Change as required

main_dir = '/Users/s1523386/Documents/year1/shape_modelling/162p'
lc_dir = main_dir+'/LC_prep/raw/'
target = '162p'

files = sorted(glob.glob(lc_dir+target+'*R.txt'))
# Check this read correctly
if len(files)==0:
    print('Problem loading LC files, check path.')
    sys.exit()
else:
    print('%i LC files found for %s.'%(len(files), target))
    print('First file in list = ', files[0])
    print('Last file in list = ', files[-1])

18 LC files found for 162p.
First file in list =  /Users/s1523386/Documents/year1/shape_modelling/162p/LC_prep/raw/162p_20070517_950_R.txt
Last file in list =  /Users/s1523386/Documents/year1/shape_modelling/162p/LC_prep/raw/162p_20180711_809_R.txt


In [3]:
# Magnitudes found in the second column of each LC file, JD in first

mjd_start = []   # First MJD of each epoch
mjd = []   # Bit inefficient, resets at end of each file
mjd_all=[]
mag = []
mag_unc=[]

for i,file in enumerate(files):
    
    f = open(file, 'r')
    lines = f.readlines()
    for line in lines:
        data = line.split()
        mjd_all.append(float(data[0]))
        mjd.append(float(data[0]))
        mag.append(float(data[1]))
        mag_unc.append(float(data[2]))
    mjd_start.append(mjd[0])
    mjd = []
    
f.close()
len(mjd_start)

18

In [4]:
# Obtain telescope codes from files filenames to parse Horizons database
siteid = []
for file in files:
    siteid.append(file[79:82])
print(siteid) # Check this is a list of 3-digit numbers 


['950', '950', '950', '309', '309', '809', '809', '309', '950', '950', '950', '071', '950', '950', '950', '809', '809', '809']


In [5]:
# Correct mid-exposure MJDs for light travel time before we obtain alphas, deltas and r_hs
# MJD_use = MJD_start-(delta/c)
au=1.496e11   #m
d2s=86400
mjd_use = []

for i,day in enumerate(mjd_start):
    obj = Horizons(id='90001062',location = siteid[i], epochs=day)
    eph = obj.ephemerides(quantities='20')
    deltcomet = float(eph['delta'])
    mjd_use.append(((day*d2s)-((deltcomet*au)/3.e8))/d2s)
mjd_use

[2454238.4504419076,
 2454239.36251031,
 2454240.3729025465,
 2456040.571266327,
 2456071.4737502034,
 2456093.490967207,
 2456096.4932989585,
 2456101.438880528,
 2457802.5503569725,
 2457803.5527208317,
 2457806.5423623323,
 2457811.504141046,
 2458219.455531825,
 2458220.4885629457,
 2458227.435531547,
 2458309.3739130935,
 2458310.374203054,
 2458311.370768656]

In [6]:
# Use light-time corrected MJDs to parse Horizons for delta, alpha, r_h
r_au = []
delta_au = []
alpha = []

for i in range(len(siteid)):
    obj = Horizons(id='90001064',location = siteid[i], epochs=mjd_use[i])
    eph = obj.ephemerides(quantities='19,20')
    
    r = float(eph['r'])
    r_au.append(r)
    
    delta = float(eph['delta'])
    delta_au.append(delta)


In [7]:
# Check the ephemerides are for 162P/Siding-Spring: it's changed id twice since I started this...
print(eph)

    targetname           datetime_str       ...      delta       delta_rate
       ---                   ---            ...        AU          km / s  
------------------ ------------------------ ... ---------------- ----------
162P/Siding Spring 2018-Jul-11 20:53:54.412 ... 4.68608624874877 26.0869665


In [8]:
# Defining funciton to calculate reduced magnitude H(1,1,alpha)
# Inputs:
#    m: apparent magnitude
#    r: heliocentric distance (AU)
#    delta: geocentric distance (AU)

# Outputs:
#    h: H(1,1,alpha)
def mag_reduce(m, r, delta):
    h = m-(5*np.log10(r*delta))
    return h

In [12]:
dir_name = main_dir+'/LC_prep/dist_corr'
if not os.path.exists(dir_name):
    os.makedirs(dir_name)

count = 0       
for i,day in enumerate(mjd_all):
    if (day in mjd_start):
        #fout.close()
        count = count+1
        fout = open(dir_name+files[count-1][64:],'w+')
        #print(count, '%.6f' % day, mag_reduce(mag[i], r_au[count-1], delta_au[count-1]), mag_unc[i])
        fout.write(str('%.6f' % day)+' '+str('%.3f' %mag_reduce(mag[i], r_au[count-1], delta_au[count-1]))+' '+str(mag_unc[i])+'\n')
    else:
        #print(count, '%.6f' % day, mag_reduce(mag[i], r_au[count-1], delta_au[count-1]), mag_unc[i])
        fout.write(str('%.6f' % day)+' '+str('%.3f' % mag_reduce(mag[i], r_au[count-1], delta_au[count-1]))+' '+str(mag_unc[i])+'\n')
fout.close()   
    
        
    

/Users/s1523386/Documents/year1/shape_modelling/162p/LC_prep/dist_corr/162p_20070517_950_R.txt


In [43]:
mjd_all

[2454238.473696,
 2454238.475177,
 2454238.47667,
 2454238.478152,
 2454238.479633,
 2454238.481115,
 2454238.482596,
 2454238.484078,
 2454238.485582,
 2454238.487064,
 2454238.549216,
 2454238.553696,
 2454238.558152,
 2454239.385819,
 2454239.387543,
 2454239.389256,
 2454239.390993,
 2454239.392705,
 2454239.399418,
 2454239.404476,
 2454239.551745,
 2454239.553469,
 2454239.601224,
 2454239.602717,
 2454239.604198,
 2454240.396273,
 2454240.397766,
 2454240.399247,
 2454240.471122,
 2454240.472604,
 2454240.474131,
 2454240.546724,
 2454240.548205,
 2454240.549687,
 2454240.607638,
 2454240.60912,
 2454240.610613,
 2456040.641995,
 2456040.643064,
 2456040.644129,
 2456040.645204,
 2456040.646293,
 2456040.647369,
 2456040.648436,
 2456040.649499,
 2456040.650576,
 2456040.65164,
 2456040.652709,
 2456040.653783,
 2456040.654872,
 2456040.655935,
 2456040.657012,
 2456040.751519,
 2456040.752621,
 2456040.753731,
 2456040.754834,
 2456040.75593,
 2456040.75705,
 2456040.758157,
 2